In [1]:
#install instructions here https://github.com/cwendt94/espn-api

# Football API
from espn_api.football import League
import pandas as pd
import numpy as np

In [11]:
# instructions for espn_s2 and swid here: https://github.com/cwendt94/espn-api/discussions/150
league = League(league_id=676821415, year=2021, 
	espn_s2='x',
	swid='y'
               )

In [3]:
teams = [str(team)[5:-1] for team in league.teams]
wins = [team.wins for team in league.teams]

In [12]:
teams

['Inside Lewan Davis',
 'King of the North',
 'Data Ballerz ',
 'A Team Lobez',
 'San Francisco Solo_1',
 'Random  Team',
 'Avellaneda Red Devils',
 'Seattle Rains',
 'California Rhinos',
 'Washington Fantasy Team']

In [5]:
## Build a dictionary with teams and their scores on each date

scores = {}

for team in teams:
	scores[team] = []

print('Current week:',league.current_week)


for week in range(1,league.current_week):
	matchups = league.scoreboard(week)
	for matchup in matchups:
		hteam = str(matchup.home_team)[5:-1]
		hscore = matchup.home_score
		ateam = str(matchup.away_team)[5:-1]
		ascore = matchup.away_score

		scores[hteam].append(hscore)
		scores[ateam].append(ascore)

Current week: 6


In [6]:
## Create a dataframe from the dictionary

df = pd.DataFrame.from_dict(scores).T
columnas = df.columns
new_columns = [columna+1 for columna in columnas]
df.columns = new_columns
df

,1,2,3,4,5
Inside Lewan Davis,86.5,150.5,107.0,139.0,109.2
King of the North,115.2,129.3,147.1,84.3,170.1
Data Ballerz,65.7,146.5,127.4,81.1,133.4
A Team Lobez,139.1,103.5,114.3,108.6,126.6
San Francisco Solo_1,124.8,156.6,99.2,92.6,117.5
Random Team,136.5,103.0,97.2,70.7,94.3
Avellaneda Red Devils,154.2,90.7,89.7,149.1,172.5
Seattle Rains,145.4,95.8,90.6,100.4,84.7
California Rhinos,91.9,78.6,133.9,128.4,171.4
Washington Fantasy Team,161.0,85.5,126.9,103.1,93.0


In [7]:
# Create a ranking of all teams in a given week by their score, in ascending order (highest score is a 9, lowest a 0)
ranks = (df.to_numpy().argsort(0)).argsort(0)

# Calculate expected wins by team based on their rank by:
#   1. Dividing the weekly rank by 9 (i.e. rank 8 means 8 out of 9 chances of winning)
#   2. Adding over all plaid weeks

expected_wins = np.round(np.sum(ranks, axis=1)/9,1)

# Calculate luck as wins minus expected wins
luck = [round(win - e_win,1) for win, e_win in zip(wins, expected_wins)]

In [8]:
output = {}

output['Teams'] = teams
output['Wins'] = wins
output['Expected Wins'] = expected_wins
output['Luck'] = luck

final_df = pd.DataFrame.from_dict(output).sort_values(by=['Luck'], ascending = False).reset_index().drop(columns=['index'])

In [9]:
final_df

,Teams,Wins,Expected Wins,Luck
0,San Francisco Solo_1,5,2.6,2.4
1,A Team Lobez,4,3.0,1.0
2,Washington Fantasy Team,3,2.4,0.6
3,Seattle Rains,2,1.7,0.3
4,King of the North,3,3.0,0.0
5,Avellaneda Red Devils,3,3.1,-0.1
6,Data Ballerz,2,2.3,-0.3
7,California Rhinos,2,2.8,-0.8
8,Random Team,0,1.4,-1.4
9,Inside Lewan Davis,1,2.7,-1.7
